In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dados = pd.read_csv("/content/df_serieb.csv") #posteriormente alterar para ler dados do banco

In [ ]:
dados = dados[dados['tournament'].isin(['Campeonato Brasileiro - Série B - 2019','Campeonato Brasileiro - Série B - 2020',\
                           'Campeonato Brasileiro - Série B - 2021','Campeonato Brasileiro - Série B - 2022',\
                           'Campeonato Brasileiro - Série B - 2023'])]

In [ ]:
dados.head()

,idgameteam,date,time,round,day,tournament,venue,idteam,team,opponent,...,yellow_cards,red_cards,two_yellow_cards,fouls_commited,fouls_draw,offsides,crosses,interceptions,tackles_won,own_goals
2280,f98930d1_1.0_2019,2019-04-26,19:15,1.0,Fri,Campeonato Brasileiro - Série B - 2019,Away,f98930d1,Bragantino,Brasil de Pelotas,...,3,0,0,23.0,11.0,0.0,16.0,20.0,10.0,0.0
2281,f98930d1_2.0_2019,2019-05-06,20:00,2.0,Mon,Campeonato Brasileiro - Série B - 2019,Home,f98930d1,Bragantino,Sport Recife,...,5,0,0,21.0,11.0,2.0,26.0,13.0,9.0,0.0
2282,f98930d1_3.0_2019,2019-05-10,19:15,3.0,Fri,Campeonato Brasileiro - Série B - 2019,Home,f98930d1,Bragantino,Atl Goianiense,...,0,0,0,22.0,16.0,0.0,12.0,13.0,16.0,0.0
2283,f98930d1_4.0_2019,2019-05-18,11:00,4.0,Sat,Campeonato Brasileiro - Série B - 2019,Away,f98930d1,Bragantino,Londrina,...,1,0,0,11.0,13.0,1.0,34.0,8.0,7.0,0.0
2284,f98930d1_5.0_2019,2019-05-21,21:30,5.0,Tue,Campeonato Brasileiro - Série B - 2019,Home,f98930d1,Bragantino,Figueirense,...,5,0,0,17.0,12.0,3.0,29.0,8.0,9.0,0.0


In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3800 entries, 2280 to 6079
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   idgameteam                 3800 non-null   object 
 1   date                       3800 non-null   object 
 2   time                       3800 non-null   object 
 3   round                      3800 non-null   float64
 4   day                        3800 non-null   object 
 5   tournament                 3800 non-null   object 
 6   venue                      3800 non-null   object 
 7   idteam                     3800 non-null   object 
 8   team                       3800 non-null   object 
 9   opponent                   3800 non-null   object 
 10  result                     3800 non-null   object 
 11  points                     3800 non-null   float64
 12  gf                         3800 non-null   int64  
 13  ga                         3800 non-null   int64  

In [ ]:
def obter_df_modelagem(df,janela):
	df = df[['round','team','tournament','opponent','result','venue','points','gf','ga','possession','gdiff','shots','p_shotsontarget','goals_shots','penalty_goals','p_penaltyconverted','shotsontarget_against','p_saves','cleansheets','penalty_attempted_against','p_penalty_saved','yellow_cards','red_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals']]
	df['cardscore'] = df['yellow_cards'] + df['red_cards']*5
	df['foulsdiff'] = df['fouls_draw'] - df['fouls_commited']
	df['tournament'] = df['tournament'].str[-4:]
	df['idteamgame'] = df['team'] + "_" + df['round'].astype(str) + "_" + df['tournament']
	df['idopponentgame'] = df['opponent'] + "_" + df['round'].astype(str) + "_" + df['tournament']

	times = df[['team','tournament','round','venue','result','idteamgame','idopponentgame']].sort_values(by=['team','tournament'])
	times = times.reset_index().rename(columns={"index":"level_2"})

	#Obtendo valores agregados em séries temporais
	rolling_stats = (df.groupby(['team','tournament'])
										.apply(lambda x: x.rolling(window=janela).agg({'points':'sum','gf':'mean','ga':'mean','possession':'mean','gdiff':'sum','shots':'mean','p_shotsontarget':'mean',\
										                                               'goals_shots':'mean','penalty_goals':'sum','p_penaltyconverted':'mean','shotsontarget_against':'mean',\
																																	 'p_saves':'mean','cleansheets':'sum','penalty_attempted_against':'sum',\
																																	 'p_penalty_saved':'mean','cardscore':'mean','foulsdiff':'mean','offsides':'mean','crosses':'mean',\
																																	 'interceptions':'mean','tackles_won':'mean','own_goals':'sum'}).shift())).reset_index()

	rolling_stats = rolling_stats.drop(['team','tournament'], axis = 1)

	#Unindo os dados
	df_modelagem = times.merge(rolling_stats, on = 'level_2')

	df_modelagem2 = df_modelagem.drop(columns=['level_2','team','tournament','round','venue','result','idopponentgame'], axis=1)

	df_modelagem2 = df_modelagem2.rename(columns={'idteamgame':'idopponentgame','points':'opp_points','gf':'opp_gf','ga':'opp_ga','possession':'opp_possession','gdiff':'opp_gdiff','shots':'opp_shots',\
									'p_shotsontarget':'opp_p_shotsontarget','goals_shots':'opp_goals_shots','penalty_goals':'opp_penalty_goals',\
									'p_penaltyconverted':'opp_p_penaltyconverted','shotsontarget_against':'opp_shotsontarget_against',\
									'p_saves':'opp_p_saves','cleansheets':'opp_cleansheets','penalty_attempted_against':'opp_penalty_attempted_against',\
									'p_penalty_saved':'opp_p_penalty_saved','cardscore':'opp_cardscore','foulsdiff':'opp_foulsdiff',\
									'offsides':'opp_offsides','crosses':'opp_crosses','interceptions':'opp_interceptions',\
									'tackles_won':'opp_tackles_won','own_goals':'opp_own_goals'})

	df_modelagem = df_modelagem.merge(df_modelagem2, on='idopponentgame')
	df_modelagem = df_modelagem.dropna()
	df_modelagem = df_modelagem[df_modelagem['possession']>0]

	df_modelagem = df_modelagem[['idteamgame','idopponentgame','tournament','venue','result','points','gf','ga','possession','gdiff','shots',\
	                             'p_shotsontarget','goals_shots','penalty_goals','p_penaltyconverted','shotsontarget_against',\
															 'p_saves','cleansheets','penalty_attempted_against','p_penalty_saved','cardscore',\
															 'foulsdiff','offsides','crosses','interceptions','tackles_won','own_goals',\
															 'opp_points','opp_gf','opp_ga','opp_possession','opp_gdiff','opp_shots','opp_p_shotsontarget',\
															 'opp_goals_shots','opp_penalty_goals','opp_p_penaltyconverted','opp_shotsontarget_against',\
															 'opp_p_saves','opp_cleansheets','opp_penalty_attempted_against','opp_p_penalty_saved','opp_cardscore',\
															 'opp_foulsdiff','opp_offsides','opp_crosses','opp_interceptions','opp_tackles_won','opp_own_goals']]

	return df_modelagem

In [ ]:
dados_modelagem = obter_df_modelagem(dados,5)

<ipython-input-6-2e78b1dbe67d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cardscore'] = df['yellow_cards'] + df['red_cards']*5
<ipython-input-6-2e78b1dbe67d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['foulsdiff'] = df['fouls_draw'] - df['fouls_commited']
<ipython-input-6-2e78b1dbe67d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [ ]:
dados_modelagem.head()

,idteamgame,idopponentgame,tournament,venue,result,points,gf,ga,possession,gdiff,...,opp_cleansheets,opp_penalty_attempted_against,opp_p_penalty_saved,opp_cardscore,opp_foulsdiff,opp_offsides,opp_crosses,opp_interceptions,opp_tackles_won,opp_own_goals
5,ABC_12.0_2023,Chapecoense_12.0_2023,2023,Home,D,4.0,0.4,1.2,0.454,-4.0,...,0.0,1.0,0.2,4.4,2.6,2.4,26.2,7.8,8.2,1.0
6,ABC_15.0_2023,Avaí_15.0_2023,2023,Away,W,3.0,0.8,1.2,0.446,-2.0,...,2.0,1.0,0.2,5.8,-5.2,0.8,19.4,12.2,8.6,0.0
7,ABC_16.0_2023,Criciúma_16.0_2023,2023,Home,L,6.0,1.2,1.0,0.414,1.0,...,2.0,0.0,0.0,2.8,-1.2,1.2,15.4,10.6,12.0,0.0
8,ABC_17.0_2023,Juventude_17.0_2023,2023,Away,D,5.0,1.2,1.2,0.462,0.0,...,1.0,0.0,0.0,4.2,-0.6,1.8,23.0,8.6,7.8,0.0
9,ABC_18.0_2023,Guarani_18.0_2023,2023,Home,L,5.0,1.2,1.2,0.456,0.0,...,1.0,0.0,0.0,4.0,0.2,1.8,13.4,7.2,9.6,0.0


In [ ]:
dados_modelagem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2070 entries, 5 to 2393
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   idteamgame                     2070 non-null   object 
 1   idopponentgame                 2070 non-null   object 
 2   tournament                     2070 non-null   object 
 3   venue                          2070 non-null   object 
 4   result                         2070 non-null   object 
 5   points                         2070 non-null   float64
 6   gf                             2070 non-null   float64
 7   ga                             2070 non-null   float64
 8   possession                     2070 non-null   float64
 9   gdiff                          2070 non-null   float64
 10  shots                          2070 non-null   float64
 11  p_shotsontarget                2070 non-null   float64
 12  goals_shots                    2070 non-null   float6

In [ ]:
dados_modelagem.columns.tolist()

['idteamgame',
 'idopponentgame',
 'tournament',
 'venue',
 'result',
 'points',
 'gf',
 'ga',
 'possession',
 'gdiff',
 'shots',
 'p_shotsontarget',
 'goals_shots',
 'penalty_goals',
 'p_penaltyconverted',
 'shotsontarget_against',
 'p_saves',
 'cleansheets',
 'penalty_attempted_against',
 'p_penalty_saved',
 'cardscore',
 'foulsdiff',
 'offsides',
 'crosses',
 'interceptions',
 'tackles_won',
 'own_goals',
 'opp_points',
 'opp_gf',
 'opp_ga',
 'opp_possession',
 'opp_gdiff',
 'opp_shots',
 'opp_p_shotsontarget',
 'opp_goals_shots',
 'opp_penalty_goals',
 'opp_p_penaltyconverted',
 'opp_shotsontarget_against',
 'opp_p_saves',
 'opp_cleansheets',
 'opp_penalty_attempted_against',
 'opp_p_penalty_saved',
 'opp_cardscore',
 'opp_foulsdiff',
 'opp_offsides',
 'opp_crosses',
 'opp_interceptions',
 'opp_tackles_won',
 'opp_own_goals']

In [ ]:
dados_modelagem.to_csv("serieb_modelagem2.csv",index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')